# Gensim - Doc2vec untuk Similarity Content
Similarity Content menggunakan vector merupakan cara sederhana untuk mendapatkan kesamaan dari sebuah artikel.
Dalam kasus ini saya akan menggunakan hasil scraping data google news indonesia berjumlah 77 documents saja.

Adapun module yang digunakan adalah menggunkan gensim

## Requirement
- Gensim 2.0

## Kode Sederhana

In [1]:
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [17]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
from pprint import pprint
import multiprocessing
import os
import re

In [18]:
dirname = 'google_news'
documents_file = os.listdir(dirname)

documents = []

for fname in documents_file:
    f = open(os.path.join(dirname,fname),'rU')
    content = f.read().decode('utf-8').lower()
    words_split = re.findall(r"[\w']+|[.,!?;]",content) # memotong kalimat berdasarkan punctuation
    
    filtered_tokens = []
    for token in words_split:
        if re.search('[a-zA-Z]', token): # filter hanya huruf saja
            filtered_tokens.append(token)
    
    title = fname.replace('.txt','')
    documents.append(TaggedDocument(filtered_tokens,[title]))
    

In [19]:
pprint(documents[:1][0].tags)

["'Formasi Tiga Bek Arsenal Belum Sempurna'"]


In [20]:
print(documents[:1][0].words[:100])

[u'arsene', u'wenger', u'menerapkan', u'formasi', u'tiga', u'bek', u'saat', u'memetik', u'kemenangan', u'atas', u'manchester', u'united', u'the', u'professor', u'sempat', u'bersikeras', u'untuk', u'tak', u'mengubah', u'skema', u'bermainnya', u'pasca', u'kekalahan', u'atas', u'crystal', u'palace', u'arsenal', u'mengubah', u'gaya', u'bermainnya', u'formasi', u'empat', u'bek', u'mulai', u'ditinggalkan', u'oleh', u'arsene', u'wenger', u'dalam', u'tiga', u'pertandingan', u'terakhir', u'wenger', u'yang', u'menerapkan', u'formasi', u'tiga', u'bek', u'membawa', u'arsenal', u'menang', u'dua', u'kali', u'dan', u'sekali', u'menelan', u'kekalahan', u'di', u'liga', u'inggris', u'kemenangan', u'terbaru', u'didapat', u'saat', u'melakoni', u'pertandingan', u'melawan', u'the', u'red', u'devils', u'skor', u'akhirnya', u'arsenal', u'menang', u'berkat', u'gol', u'yang', u'dibukukan', u'oleh', u'granit', u'xhaka', u'dan', u'danny', u'welbeck', u'wenger', u'menilai', u'bahwa', u'tim', u'besutannya', u'bisa'

In [21]:
cores = multiprocessing.cpu_count()
model = Doc2Vec(dm=0, dbow_words=1, size=100, window=8, min_count=20, iter=100, workers=cores, sample=1e-4, negative=2)

In [22]:
model.scan_vocab(documents,update=False)
print(str(model))

Doc2Vec(dbow+w,d100,n2,w8,mc20,s0.0001,t4)


In [23]:
model.build_vocab(documents,update=False)
print(str(model))

Doc2Vec(dbow+w,d100,n2,w8,mc20,s0.0001,t4)


In [24]:
%time model.train(documents, total_examples=model.corpus_count, epochs=model.iter)

CPU times: user 11.8 s, sys: 1.27 s, total: 13.1 s
Wall time: 5.2 s


1314523

In [25]:
print(str(model))
pprint(model.docvecs.most_similar(positive=["5 Kesamaan Xiaomi Mi 6 dan iPhone 7 Plus"], topn=10))

Doc2Vec(dbow+w,d100,n2,w8,mc20,s0.0001,t4)
[('Ini Alasan Xiaomi Mi 6 Tiru iPhone 7', 0.8043391704559326),
 ('Mengapa Xiaomi Mi 6 Harus Mengikuti Langkah iPhone 7? - Kompas.com',
  0.7488471269607544),
 ('Resmi Masuk Negara Turki, iPhone 7 Tembus Harga Rp 16 Jutaan Dan Menjadi Rekor Harga Tertinggi Hingga Saat Ini',
  0.5795214772224426),
 ('Qualcomm Sudah Daftarkan Snapdragon 845 di Website Resminya',
  0.5662438869476318),
 ('Smartphone Baru ASUS X00ID Muncul di GFXBench dengan Dual-camera',
  0.5153516530990601),
 ('Samsung Galaxy J7 (2017) Muncul Lagi di GFXBench dengan Spesifikasi Berbeda',
  0.4440805912017822),
 ('Negara Mana yang Jual iPhone 7 Termahal dan Termurah?', 0.4436926543712616),
 ('Negara Ini Jual iPhone 7 dengan Harga Paling Mahal di Dunia',
  0.440419465303421),
 ('10 Hubungan keluarga para seleb Bollywood ini jarang diketahui',
  0.43464940786361694),
 ('ASUS Zenfone X015D Muncul di GFXBench Usung Chipset MediaTek dan RAM 3GB',
  0.42377620935440063)]


In [28]:
text_search = '''Soal Reklamasi Teluk Jakarta, Bang Sandi Bakal Ngobrol Bareng dengan Luhut'''
inferred_vector = model.infer_vector(text_search.lower().split())
model.docvecs.most_similar([inferred_vector], topn=10)

[('Sandiaga Uno Tanggapi Pernyataan Luhut Pandjaitan soal Reklamasi',
  0.5000295639038086),
 ('Soal Reklamasi Teluk Jakarta, Bang Sandi Bakal Ngobrol Bareng dengan Luhut',
  0.4931764304637909),
 ('Sandiaga Sambut Tantangan Menko Luhut Bicara soal Reklamasi - VIVA.co.id',
  0.47335365414619446),
 ('Jawab Tantangan Luhut Soal Reklamasi, Sandi Akan Siapkan Tim',
  0.4121737778186798),
 ('Rinni Wulandari dan Jevin Julian Pamer Buku Nikah', 0.241092711687088),
 ('Sosok Budidarmo Bagi Tito Karnavian', 0.20594513416290283),
 ('Sidang Korupsi e-KTP, Pengacara Hotma Sitompul Dipanggil Jadi Saksi',
  0.20400218665599823),
 ('BMKG: Tidak Mungkin Salju Turun di Jakarta', 0.1884603500366211),
 ('Petani Dihalangi Paspampres di Depannya, Ini yang Dilakukan Jokowi',
  0.1664963960647583),
 ('Polda Riau Bentuk Tim Usut Pemerasan di Rutan Pekanbaru',
  0.16344894468784332)]

In [27]:
model.docvecs['5 Kesamaan Xiaomi Mi 6 dan iPhone 7 Plus']

array([ 0.55978322,  0.56031793,  0.05888164,  0.42614335, -0.83653086,
       -0.34832302,  0.35508612, -0.14448757,  0.26461178,  0.23192292,
       -0.79622906,  0.74073577, -0.57028621, -0.37256494,  0.48538208,
       -0.46242538,  0.28113458,  0.18262506, -0.08646963,  0.19478701,
        0.08811761, -1.08414376,  0.16470033, -0.1053811 ,  0.73071486,
       -0.62379593,  0.34942403, -0.60945785, -0.03622055,  0.11027819,
        0.41261092, -0.26597378,  0.14532451,  0.59876686, -0.25098893,
        0.9371357 , -0.2655668 , -0.49870977,  1.41683602,  0.0855694 ,
       -0.61547297,  0.93351853,  0.40202183, -0.67395622,  0.41190007,
        0.35688952,  0.31151399,  0.30218858,  0.03822093, -0.57568526,
        0.27861571, -0.28015926,  0.16584191, -0.63383502, -0.7314384 ,
        0.49057713, -0.39977276,  0.95082641,  0.78501958,  0.22631977,
       -0.06571314, -0.25181508,  0.52627206, -0.19543737,  0.63155901,
        0.00809581, -0.49587014, -0.50795776, -0.39754882,  0.31